In [38]:
from typing_extensions import TypedDict, Literal, Annotated
from typing import List
from langgraph.types import Send
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from pydantic import BaseModel
from operator import add

llm = init_chat_model("openai:gpt-4o")

In [39]:
class State(TypedDict):

    document: str
    final_summary: str
    summaries: Annotated[list[dict], add]

In [40]:
def summarize_p(args):
    paragraph = args["paragraph"]
    index = args["index"]
    response = llm.invoke(
        f"Write a 3-sentence summary for this paragraph: {paragraph}",
    )
    return {
        "summaries": [
            {
                "summary": response.content,
                "index": index,
            }
        ],
    }


def dispatch_summarizers(state: State):
    chunks = state["document"].split("\n\n")
    return [
        Send("summarize_p", {"paragraph": chunk, "index": index})
        for index, chunk in enumerate(chunks)
    ]


def final_summary(state: State):
    response = llm.invoke(
        f"Using the following summaries, give me a final one {state["summaries"]}"
    )
    return {
        "final_summary": response.content,
    }

In [41]:
graph_builder = StateGraph(State)

graph_builder.add_node("summarize_p", summarize_p)
graph_builder.add_node("final_summary", final_summary)


graph_builder.add_conditional_edges(
    START,
    dispatch_summarizers,
    ["summarize_p"],
)

graph_builder.add_edge("summarize_p", "final_summary")
graph_builder.add_edge("final_summary", END)


graph = graph_builder.compile()

In [42]:
with open("fed_transcript.md", "r", encoding="utf-8") as file:
    document = file.read()


for chunk in graph.stream(
    {"document": document},
    stream_mode="updates",
):
    print(chunk, "\n")

{'summarize_p': {'summaries': [{'summary': 'Our monetary policy is directed by a dual mandate focused on achieving maximum employment and stable prices. This means we aim to create conditions that support job growth while controlling inflation. These objectives are central to fostering economic stability and prosperity for the American public.', 'index': 13}]}} 

{'summarize_p': {'summaries': [{'summary': 'Labor demand has decreased, indicating a softening job market. The current rate of job creation is insufficient to maintain a stable unemployment rate. As a result, unemployment is likely to rise if job growth does not accelerate.', 'index': 7}]}} 

{'summarize_p': {'summaries': [{'summary': 'The labor market is experiencing an unusual slowdown in the supply of and demand for workers. This less dynamic market has increased the downside risks for employment. The projected unemployment rate is 4.5 percent by the end of the year, with a gradual decrease expected thereafter.', 'index': 9